In [110]:
!pip install pyLDAvis 

In [123]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nhgirma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [124]:
model_df = pd.read_csv('clean_processed_tweet_data.csv')
model_df = model_df.fillna("")
model_df.head()

,created_at,statuses_count,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries
0,2022-08-07 22:31:20+00:00,8097,"<a href=""http://twitter.com/download/android"" ...",RT @i_ameztoy: Extra random image (I):\n\nLets...,\n\nLets focus in one very specific zone of th...,-1,0.000000e+00,0.08125,en,15760,2,i_ameztoy,232,20497,2621,,City,,,
1,2022-08-07 22:31:16+00:00,5831,"<a href=""http://twitter.com/download/android"" ...",RT @IndoPac_Info: #China's media explains the ...,#China's media explains the military reasons ...,0,-1.000000e-01,0.10000,en,6967,201,ZIisq,3,65,272,,China,,,
2,2022-08-07 22:31:07+00:00,1627,"<a href=""http://twitter.com/download/android"" ...","China even cut off communication, they don't a...","China even cut off communication, they don't a...",-1,0.000000e+00,0.00000,en,2166,0,Fin21Free,0,85,392,,XiJinping,,Netherlands,
3,2022-08-07 22:31:06+00:00,1627,"<a href=""http://twitter.com/download/android"" ...","Putin to #XiJinping : I told you my friend, Ta...","Putin to #XiJinping : I told you my friend, Ta...",1,1.000000e-01,0.35000,en,2166,0,Fin21Free,0,85,392,,XiJinping,,Netherlands,
4,2022-08-07 22:31:04+00:00,18958,"<a href=""http://twitter.com/download/iphone"" r...","RT @ChinaUncensored: I’m sorry, I thought Taiw...","I’m sorry, I thought Taiwan was an independen...",0,-6.938894e-18,0.55625,en,17247,381,VizziniDolores,0,910,2608,,,,"Ayent, Schweiz",


In [125]:
model_df.shape

(22000, 20)

In [126]:
sentiment_data = model_df.copy(deep=True)
sentiment_data.drop(sentiment_data[sentiment_tweet_data['sentiment'] == -1].index, inplace=True)
sentiment_data.reset_index(drop=True, inplace=True)
train_set = sentiment_tweet_data.iloc[:17600,]
test_set = sentiment_tweet_data.iloc[17601:,]

In [127]:
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
unigram_vectorizer.fit(train_set['clean_text'].values)

CountVectorizer()

In [128]:
X_train_unigram = unigram_vectorizer.transform(train_set['clean_text'].values)

In [129]:
unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)

TfidfTransformer()

In [130]:
X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)

In [131]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
bigram_vectorizer.fit(train_set['clean_text'].values)

CountVectorizer(ngram_range=(1, 2))

In [132]:
X_train_bigram = bigram_vectorizer.transform(train_set['clean_text'].values)

In [133]:
bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)

TfidfTransformer()

In [134]:
X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)

In [135]:
def train_and_show_scores(X: csr_matrix, y: np.array, title: str) -> None:
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y,train_size=0.75, stratify=y
    )

    clf = SGDClassifier()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    valid_score = clf.score(X_valid, y_valid)

    global_vars = globals()
    if(valid_score > global_vars['best_score']):
        global_vars['best_model'] = clf
        global_vars['best_model_name'] = title
        global_vars['best_score'] = valid_score

    print(f'{title}\nTrain score: {round(train_score, 2)} ; Validation score: {round(valid_score, 2)}\n')

In [136]:
y_train = train_set['sentiment'].values
y_train

array([-1,  0, -1, ..., -1, -1,  1])

In [137]:
best_model = ""
best_model_name = ""
best_score = 0

train_and_show_scores(X_train_unigram, y_train, 'Unigram Counts')
train_and_show_scores(X_train_unigram_tf_idf, y_train, 'Unigram Tf-Idf')
train_and_show_scores(X_train_bigram, y_train, 'Bigram Counts')
train_and_show_scores(X_train_bigram_tf_idf, y_train, 'Bigram Tf-Idf')

Unigram Counts
Train score: 1.0 ; Validation score: 0.94

Unigram Tf-Idf
Train score: 0.99 ; Validation score: 0.93

Bigram Counts
Train score: 1.0 ; Validation score: 0.93

Bigram Tf-Idf
Train score: 0.99 ; Validation score: 0.92



In [138]:
print(f'Best Model: {best_model_name} \nValidation Score: {round(best_score, 2)}')


Best Model: Unigram Counts 
Validation Score: 0.94


In [139]:
sgd = joblib.dump(best_model, './trained_model/newsentimentSGDmodel.jl')

In [140]:
from nltk.corpus import stopwords
stop_words= stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [141]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
model_df['clean_text'] = df['clean_text'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))


In [142]:
class PrepareData:
      def __init__(self,df):
        self.df=df
    
      def preprocess_data(self):
        tweets_df = self.df.loc[self.df['lang'] =="en"]

    
    #text Preprocessing
        model_df['clean_text']=model_df['clean_text'].astype(str)
        model_df['clean_text'] = model_df['clean_text'].apply(lambda x: x.lower())
        model_df['clean_text']= model_df['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))

        #Converting tweets to list of words For feature engineering
        sentence_list = [tweet for tweet in model_df['clean_text']]
        word_list = [sent.split() for sent in sentence_list]
        # print(word_list)

        #Create dictionary which contains Id and word 
        word_to_id = corpora.Dictionary(word_list) #generate unique tokens
        #  we can see the word to unique integer mapping
        # print(word_to_id.token2id)
        # using bag of words(bow), we create a corpus that contains the word id and its frequency in each document.
        corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]
        

        return word_list, word_to_id, corpus_1


In [143]:
PrepareData_obj=PrepareData(model_df)
word_list ,id2word,corpus=PrepareData_obj.preprocess_data()

In [144]:
model_df["clean_text"]

0        let focu in one veri specif zone of the wester...
1        china media explain the militari reason for ea...
2        china even cut off commun they dont anwer phon...
3        putin to xijinp i told you my friend taiwan wi...
4        i’m sorri i thought taiwan wa an independ coun...
                               ...                        
21995    a good infograph of china missil launch on tai...
21996    a good infograph of china missil launch on tai...
21997                       reuter thank pelosi smart move
21998    taiwan peopl desir for unif with china near re...
21999    taiwan peopl desir for unif with china near re...
Name: clean_text, Length: 22000, dtype: object

In [145]:
print(corpus[0:3])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(5, 2), (8, 1), (13, 5), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)], [(13, 2), (18, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1)]]


In [146]:
id_words = [[(id2word[id], count) for id, count in line] for line in corpus]

In [147]:
print(id_words[0:3])

[[('citi', 1), ('coast', 1), ('district', 1), ('focu', 1), ('gt', 1), ('in', 1), ('let', 1), ('longj', 1), ('of', 1), ('one', 1), ('specif', 1), ('taichung', 1), ('ta…', 1), ('the', 1), ('veri', 1), ('western', 1), ('zone', 1)], [('in', 2), ('of', 1), ('the', 5), ('area', 1), ('china', 1), ('drill', 1), ('each', 1), ('explain', 1), ('for', 1), ('label', 1), ('media', 1), ('militari', 1), ('pi…', 1), ('read', 1), ('reason', 1), ('strait', 1), ('taiwan', 1)], [('the', 2), ('china', 1), ('anwer', 1), ('ask', 1), ('but', 1), ('chang', 1), ('clown', 1), ('commun', 1), ('cut', 1), ('dont', 1), ('enter', 1), ('even', 1), ('from', 1), ('here', 1), ('mind', 1), ('off', 1), ('phonecal', 1), ('putin', 1), ('stage', 1), ('they', 1), ('to', 2), ('us', 1), ('xijinp', 1), ('zelenskyyua', 1)]]


In [148]:
type(id_words)

list

In [149]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [150]:
pprint(lda_model.print_topics())

[(0,
  '0.044*"is" + 0.041*"of" + 0.035*"taiwan" + 0.034*"it" + 0.033*"thi" + '
  '0.027*"by" + 0.022*"the" + 0.021*"presid" + 0.019*"out" + 0.018*"head"'),
 (1,
  '0.023*"an" + 0.023*"video" + 0.018*"travel" + 0.017*"russia" + 0.017*"dead" '
  '+ 0.016*"hotel" + 0.016*"found" + 0.015*"watch" + 0.014*"independ" + '
  '0.014*"on"'),
 (2,
  '0.056*"the" + 0.045*"to" + 0.040*"taiwan" + 0.026*"and" + 0.024*"is" + '
  '0.023*"a" + 0.019*"china" + 0.017*"pelosi" + 0.017*"of" + 0.016*"in"'),
 (3,
  '0.023*"to" + 0.023*"visit" + 0.021*"democrat" + 0.021*"stop" + 0.019*"her" '
  '+ 0.019*"speaker" + 0.018*"show" + 0.018*"not" + 0.016*"we" + 0.016*"amp"'),
 (4,
  '0.054*"taiwan" + 0.046*"the" + 0.038*"china" + 0.035*"on" + 0.027*"in" + '
  '0.027*"of" + 0.016*"chines" + 0.016*"missil" + 0.016*"island" + 0.016*"a"')]


In [151]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('is', 0.044000532),
   ('of', 0.040531408),
   ('taiwan', 0.03487833),
   ('it', 0.034034688),
   ('thi', 0.033112794),
   ('by', 0.027281532),
   ('the', 0.02222499),
   ('presid', 0.020650964),
   ('out', 0.019398473),
   ('head', 0.018423075)]),
 (1,
  [('an', 0.023177875),
   ('video', 0.023113232),
   ('travel', 0.018181002),
   ('russia', 0.01702097),
   ('dead', 0.016761908),
   ('hotel', 0.016320212),
   ('found', 0.01601878),
   ('watch', 0.015274936),
   ('independ', 0.014375682),
   ('on', 0.014048988)]),
 (2,
  [('the', 0.05628892),
   ('to', 0.045252256),
   ('taiwan', 0.0397211),
   ('and', 0.026164126),
   ('is', 0.024309853),
   ('a', 0.023368152),
   ('china', 0.018679872),
   ('pelosi', 0.0174979),
   ('of', 0.01679867),
   ('in', 0.016422248)]),
 (3,
  [('to', 0.022967117),
   ('visit', 0.022582497),
   ('democrat', 0.021309456),
   ('stop', 0.0206835),
   ('her', 0.018991478),
   ('speaker', 0.018586267),
   ('show', 0.017990032),
   ('not', 0.017944494),
 

In [152]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -7.361043132861787

 Ldamodel Coherence Score/Accuracy on Tweets:  0.2951545357696369


In [154]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.174380 -0.044520       1        1  43.235056
4     -0.196279  0.059023       2        1  27.739217
0     -0.117063  0.014979       3        1  10.570076
3      0.221255 -0.297075       4        1   9.836626
1      0.266466  0.267593       5        1   8.619024, topic_info=        Term         Freq         Total Category  logprob  loglift
113       is  6521.000000   6521.000000  Default  30.0000  30.0000
8         of  7945.000000   7945.000000  Default  29.0000  29.0000
59        it  3665.000000   3665.000000  Default  28.0000  28.0000
95       thi  2745.000000   2745.000000  Default  27.0000  27.0000
102       on  5499.000000   5499.000000  Default  26.0000  26.0000
...      ...          ...           ...      ...      ...      ...
534    thing   331.589135    388.903923   Topic5  -4.6835   2.2918
102       on   503.799885   5499.429102   Topic5  -4.2652   0.0610
1014  ukrain   321.092788   1078.319139   Topic5  -4.7157   1.2398
30    taiwan   357.937309  15313.874486   Topic5  -4.6070  -1.3050
18     china   315.521594   8342.556562   Topic5  -4.7332  -0.8237

[260 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
265       2  0.998822     3
162       2  0.998172   4th
53        1  0.680716     a
53        2  0.291134     a
53        4  0.028174     a
...     ...       ...   ...
1202      3  0.996861  year
1410      3  0.996540    yi
74        1  0.979588   you
74        3  0.020298   you
882       4  0.998968     😈

[278 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 4, 2])